In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [34]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # input 1 channel, output 6 channel, kernel size 3*3
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        
        self.fc1 = nn.Linear(14*14*32, 128)  # 6*6 from image dimension
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # Flatten to vector to input the neural network
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [35]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=6272, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [36]:
params = list(net.parameters())

In [38]:
params[0].size()

torch.Size([32, 1, 3, 3])

In [39]:
params[1].size()

torch.Size([32])

In [53]:
input = torch.randn(1, 1, 28, 28)
out_call = net(input)
out_forward = net.forward(input)

out_call == out_forward

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


tensor([[True, True, True, True, True, True, True, True, True, True]])

In [31]:
net = Net()
def print_info(self, input, output):
    # input is a tuple of packed inputs
    # output is a Tensor. output.data is the Tensor we are interested
    print('Inside ' + self.__class__.__name__ + ' forward')
    
    print('')
    print('input: ', type(input), ', len: ', len(input))
    print('input[0]: ', type(input[0]), ', shape: ', input[0].shape)
    print('output: ', type(output), ', len: ', len(output), output.data.shape)
    

net.conv2.register_forward_hook(print_info)

out = net(input)

Inside Conv2d forward

input:  <class 'tuple'> , len:  1
input[0]:  <class 'torch.Tensor'> , shape:  torch.Size([1, 32, 28, 28])
output:  <class 'torch.Tensor'> , len:  1 torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])


In [52]:

target = torch.tensor([3])
loss_fn = nn.CrossEntropyLoss()

net = Net()
def print_backward_info(self, grad_input, grad_output):
    print('Inside ' + self.__class__.__name__ + ' backward')
    print('grad_input: ', type(grad_input), ', len: ', len(grad_input))
    print('grad_output: ', type(grad_output), ', len: ', len(grad_output))
    print('grad_output[0]: ', type(grad_output[0]), ', size: ', grad_output[0].shape)


net.conv1.register_backward_hook(print_backward_info)

out = net(input)
err = loss_fn(out, target)
err.backward()

Inside Conv2d backward
grad_input:  <class 'tuple'> , len:  3
grad_output:  <class 'tuple'> , len:  1
grad_output[0]:  <class 'torch.Tensor'> , size:  torch.Size([1, 32, 28, 28])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [ ]:
def __call__(self, *input, **kwargs):
  for hook in self._forward_pre_hooks.values():
    hook(self, input)
  
  result = self.forward(*input, **kwargs)
  
  for hook in self._forward_hooks.values():
    hook(self, input, result)
    # TODO
  
  for hook in self._backward_hooks.values():
    # TODO

  return result